# Introduction
- This introduction is the easiest way for Python folks to play with Percolate
- We provider the Postgres client using the Docker compose connection parameteres as defaults
- We can learn how to register entities after which everyone else becomes possible

In [5]:
import percolate
from percolate.services import PostgresService
pg = PostgresService()

In [8]:
def bootstrap(root='../../../extension/sql'):
    """util to generate the sql that we use to setup percolate"""
    
    from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function, Session, Dialogue
    from percolate.models.p8 import sample_models
    from percolate.models.utils import SqlModelHelper
    import glob

    root = root.rstrip('/')
        
    """compile the functions into one file"""
    with open(f'{root}/01_add_functions.sql', 'w') as f:
        for sql in glob.glob('../../../extension/sql-staging/p8_pg_functions/**/*.sql',recursive=True):
            with open(sql, 'r') as sql:
                f.write(sql.read())
                f.write('\n\n---------\n\n')

    """add base tables"""            
    with open(f'{root}/02_create_primary.sql', 'w') as f:
        for model in [ Project, Agent, ModelField, LanguageModelApi, Function, Session, Dialogue]:
            f.write(pg.repository(model).model_registration_script(secondary=False, primary=True))

    """add the rest"""
    with open(f'{root}/03_create_secondary.sql', 'w') as f:    
        for model in [ Project, Agent, ModelField, LanguageModelApi, Function, Session, Dialogue]:
            f.write(pg.repository(model).model_registration_script(secondary=True, primary=False))
        script = SqlModelHelper(LanguageModelApi).get_data_load_statement(sample_models)
        f.write('\n\n-- -----------\n')
        f.write('-- sample models--\n\n')
        
        f.write(script)
bootstrap()

## Registering entities which are Agents is the key way to use Percolate
- entity models describe agents completely
- Percolate assumes agents are serializable in the "Agent" model which has structured response, sytem prompt and external function refs
- We register Entities as tables that can store data, as Agents that can allow interaction with data
- Agents are added as structured tables but other indexes are added too; embeddings on fields that need them and graph node registry

In [2]:
from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function, Session, Dialogue

#we register the core models - these are added by scripts in install anyway but it illustrates for 'bring your own'
for model in [Dialogue]:
    repo = pg.repository(model)
    repo.register(register_entities=True)

2025-01-25 14:14:20.386 | WARNING  | percolate.services.PostgresService:execute:171 - Failing to execute query model <class 'percolate.models.p8.types.Dialogue'> - verbose_errors=False - relation "Dialogue" already exists

2025-01-25 14:14:20.388 | WARNING  | percolate.services.PostgresService:register:94 - The table already exists - will check for schema migration or ignore
2025-01-25 14:14:20.492 | WARNING  | percolate.services.PostgresService:register:98 - Migrating schema with ALTER TABLE p8."Dialogue" ADD COLUMN tool_calls JSON; ALTER TABLE p8."Dialogue" ADD COLUMN tool_eval_data JSON; 
2025-01-25 14:14:20.587 | WARNING  | percolate.services.PostgresService:execute:171 - Failing to execute query model <class 'percolate.models.p8.types.Dialogue'> - verbose_errors=False - relation "p8_Dialogue_embeddings" already exists

2025-01-25 14:14:20.588 | WARNING  | percolate.services.PostgresService:register:107 - The embedding-associated table already exists
2025-01-25 14:14:20.588 | DEBUG

## Percolate configures Langauge models and assumes we can load tokens from the env
- While not recommended in production, for simplicity you might add tokens locally in your database
- The example below registers unique model names - the name is sometimes of the from provider-model if you want to select an API

In [ ]:
from percolate.utils import make_uuid
import os


fetched=pg.repository(LanguageModelApi).update_records(models)